In [1]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("assets_viejos/native_contact5series.dat",sep="\s+") 
# df = df.set_index('#Frame')

In [4]:
nn = pd.read_csv("assets_viejos/nonnative_contact5series-002.dat",sep="\s+") 
# nn = nn.set_index('#Frame')

In [ ]:
cc = df.merge(nn, on='#Frame')
cc = cc.set_index('#Frame')

In [ ]:
df.columns.shape,nn.columns.shape, cc.columns.shape

In [ ]:
#Definimos las subpartes de analisis de e2012  y de la cavidad
imidazol = ["@N2_", "@C10_", "@N3_", "@C23_", "@C22_","@H9_", "@H20_", "@C25_", "@H24_", "@H25_", "@H26_"]
metoxi = ["@C20_", "@C8_", "@C19_", "@C17_", "@C5_", "@C9_", "@O2_", "@C24_", "@H21_", "@H22_", "@H23_", "@H19_", "@H5_", "@H8_"]
piper = ["@C14_", "@C15_", "@N1_", "@C11_", "@C12_", "@C1_", "@O1_", "@H10_", "@H11_", "@H12_", "@H13_", "@H1_", "@H2_", "@C3_", "@H3_", "@C13_", "@H14_"]
metil = ["@C13_", "@H14_", "@C16_", "@H15_", "@H16_", "@H17_"]
fluoro = ["@C2_", "@C4_", "@C6_", "@C21_", "@C7_", "@C18_", "@H4_", "@H6_", "@F1_", "@H7_", "@H18_"]
partes_e2012 = [imidazol,metoxi,piper,metil,fluoro]
cav= ["210","211","701", "702", "706", "707","708","709","710","711","712","713", "769", "772","773","774", "776", "777", "780", "827","828","829","830","831","832","833", "835","836","1332","1333","1334","1335", "1338","1339","1340", "1342","1343","1344","1345","1346"]
cav_chido =['NCST-ILE-242', 'NCST-ASN-243', 'PSEN-PHE-105', 'PSEN-TYR-106', 'PSEN-ASP-110', 'PSEN-GLY-111','PSEN-GLN-112', 'PSEN-LEU-113', 'PSEN-ILE-114', 'PSEN-TYR-115', 'PSEN-THR-116', 'PSEN-PRO-117','PSEN-LEU-173', 'PSEN-PHE-176', 'PSEN-PHE-177', 'PSEN-SER-178', 'PSEN-ILE-180', 'PSEN-TYR-181','PSEN-GLU-184', 'PSEN-ALA-231', 'PSEN-LEU-232', 'PSEN-MET-233', 'PSEN-ALA-234', 'PSEN-LEU-235','PSEN-VAL-236', 'PSEN-PHE-237', 'PSEN-LYS-239', 'PSEN-TYR-240','AB-GLY-25', 'AB-SER-26','AB-ASN-27', 'AB-LYS-28', 'AB-ILE-31', 'AB-ILE-32','AB-GLY-33', 'AB-MET-35', 'AB-VAL-36', 'AB-GLY-37', 'AB-GLY-38','AB-VAL-39']

#Primero vamos a separar el df de contactos en cada una de las subpartes
cols = [[] for i in partes_e2012]
c = -1
for partes in partes_e2012:
    c+=1
    for atom in partes:
        for  col in df.columns.values:
            if atom in col:
                cols[c].append(col)
                

In [ ]:
#Ahora vamos a separar por subpartes y por cada aminoacido, asi podemos ver con quien si esta interactuando
cols_amino = [[[] for i in cav] for j in partes_e2012]
c = -1
for partes in cols:              #Para cada parte en e2012
    c+=1
    c2=-1
    for aa in cav:               #checar cada aminoacido
        c2+=1
        for atom in partes:     #si hay un contacto o no 
            if aa in atom:
                cols_amino[c][c2].append(atom)

#Hagamos la sumas de los contactos
contactos = [[[] for i in cav] for j in partes_e2012]
c = -1
for i in  cols_amino:
    c+=1
    c2=-1
    for amino in i:
        c2+=1
        if len(amino)==0:
            contactos[c][c2].append(0)
        else:
#             contactos[c][c2].append(np.mean(np.sum(df[cols_amino[c][c2]].values, axis=1)/len(cols_amino[c][c2])))
            contactos[c][c2].append(np.mean(np.sum(df[cols_amino[c][c2]].values, axis=1)))

In [ ]:
contactos = np.array(contactos).reshape(5,40)

fig2 = go.Figure()
fig2.add_traces(go.Heatmap(z=contactos.flatten(),
                          x=cav_chido*5,
                          y=np.array([[i]*len(cav) for i in["Imidazol","Metoxibenceno","Piperidona","Metilo","Fluorobencilo"]]).flatten(),
                          colorscale='OrRd'))

fig2.update_layout(title='Matriz de contactos <br> E2012-Cavidad',title_x=0.5)
fig2.data[0].colorbar.title.text = "Núm. contactos normalizados"
fig2.data[0].colorbar.titleside = "right"
fig2.update_xaxes(title="Cavidad",tickangle=-90)
fig2.update_yaxes(title="E2012")

In [ ]:
maa = -3
maximos_cont = np.sort(contactos,axis=1)[:,maa:]   #Obtiene los maa residuos con los que esta mas cerca
maximos_cont = np.fliplr(maximos_cont)

In [ ]:
ys =[]
for i in maximos_cont:                          #Vamos a obtener el indice de cada contacto maximo
    for j in i:                         
        x,y=(np.where(contactos==j))
        ys.append(y[0])
        
#Son el id y el resname de los residuos que tienen mas cera
res_min_cav =np.array([cav_chido[i] for i in ys]).reshape(5,maa)

In [ ]:
#Hagamos la sumas de los contactos
contactos_maximos = [[[] for i in range(abs(maa))] for j in partes_e2012]
ys = np.array(ys).reshape(5,maa)
c = -1
for i in  cols_amino:
    c+=1
    c2=-1
    for amino_max in ys[c]:
        c2+=1
        contactos_maximos[c][c2].append(np.sum(df[i[amino_max]].values, axis=1)/len(i[amino_max]))

contactos_maximos = np.squeeze(np.array(contactos_maximos))

In [ ]:
viol = go.Figure()
for i in range(abs(maa)):
    viol.add_traces(go.Violin(y=contactos_maximos[0][i],x=[res_min_cav[0][i]]*df.shape[0],
                             line_color="indianred",
                             meanline_visible=True,))
viol